# Kubernetes lab

First move to the source directory where you already downloaded the git repo sources:

git clone https://github.com/mjbright/LinuxConEU-ContainerOrchestration

In [1]:
pwd

/home/group20/2016-Oct-31/LinuxConEU-ContainerOrchestration


In [2]:
git remote -v

origin	https://github.com/mjbright/LinuxConEU-ContainerOrchestration (fetch)
origin	https://github.com/mjbright/LinuxConEU-ContainerOrchestration (push)


In [3]:
cd labs

In [4]:
ls

playbook.k8s  playbook.mesos  roles  Vagrantfile.multi	Vagrantfile.single


We first create symbolic links to use the appropriate playbook and Vagrantfile

In [5]:
ln -s playbook.k8s       playbook.yml
ln -s Vagrantfile.single Vagrantfile

In [6]:
ls -al

total 28
drwxrwxr-x. 3 group20 group20 4096 Oct 31 11:47 .
drwxrwxr-x. 9 group20 group20 4096 Oct 31 11:46 ..
-rw-rw-r--. 1 group20 group20  265 Oct 31 10:47 playbook.k8s
-rw-rw-r--. 1 group20 group20  165 Oct 31 10:47 playbook.mesos
lrwxrwxrwx. 1 group20 group20   12 Oct 31 11:47 playbook.yml -> playbook.k8s
drwxrwxr-x. 9 group20 group20 4096 Oct 31 10:47 roles
lrwxrwxrwx. 1 group20 group20   18 Oct 31 11:47 Vagrantfile -> Vagrantfile.single
-rw-rw-r--. 1 group20 group20 1956 Oct 31 10:47 Vagrantfile.multi
-rw-rw-r--. 1 group20 group20 1720 Oct 31 10:47 Vagrantfile.single


# Create our lab environment using Vagrant

Now we have selected which Vagrantfile we use we can let Vagrant startup our virtual machine.

The
```shell
vagrant up```
command will take several minutes to complete and will invoke ansible to provision Kubernetes on our node as show below:

In [7]:
vagrant up

Bringing machine 'node1' up with 'virtualbox' provider...
==> node1: Box 'centos/7' could not be found. Attempting to find and install...
    node1: Box Provider: virtualbox
    node1: Box Version: >= 0
==> node1: Loading metadata for box 'centos/7'
    node1: URL: https://atlas.hashicorp.com/centos/7
==> node1: Adding box 'centos/7' (v1609.01) for provider: virtualbox
    node1: Downloading: https://atlas.hashicorp.com/centos/boxes/7/versions/1609.01/providers/virtualbox.box
==> node1: Successfully added box 'centos/7' (v1609.01) for 'virtualbox'!
==> node1: Importing base box 'centos/7'...
==> node1: Matching MAC address for NAT networking...
==> node1: Checking if box 'centos/7' is up to date...
==> node1: Setting the name of the VM: labs_node1_1477910979585_72722
==> node1: Clearing any previously set network interfaces...
==> node1: Preparing network interfaces based on configuration...
    node1: Adapter 1: nat
==> node1: Forwarding ports...
    node1: 80 (guest) => 8080 (host) (

Let's use

```shell
vagrant status
```
to show the status of the virtual machine

In [8]:
vagrant status

Current machine states:

node1                     running (virtualbox)

The VM is running. To stop this VM, you can run `vagrant halt` to
shut it down forcefully, or you can run `vagrant suspend` to simply
suspend the virtual machine. In either case, to restart it again,
simply run `vagrant up`.


We can now use the

```shell
vagrant ssh
```
command to connect to that node, as an interactive session, to to run a specific command as below:

In [9]:
vagrant ssh node1 -c "ls -al /tmp/" 

total 16
drwxrwxrwt.  7 root root 4096 Oct 31 10:54 .
dr-xr-xr-x. 18 root root 4096 Oct 31 10:50 ..
drwxrwxrwt.  2 root root    6 Oct  3 13:39 .font-unix
drwxrwxrwt.  2 root root    6 Oct  3 13:39 .ICE-unix
-rwx------.  1 root root 2326 Oct  3 13:48 ks-script-nDh093
-rwx------.  1 root root  827 Oct  3 13:49 ks-script-SstNaF
drwxrwxrwt.  2 root root    6 Oct  3 13:39 .Test-unix
drwxrwxrwt.  2 root root    6 Oct  3 13:39 .X11-unix
drwxrwxrwt.  2 root root    6 Oct  3 13:39 .XIM-unix
-rw-------.  1 root root    0 Oct  3 13:34 yum.log



On your master host, check out the kubernetes git repository, and change into the guestbook directory:

**Connect to the virtual machine using**
vagrant ssh node1


The following commands are all to be run within the virtual machine

Note: This lab was run with revision ddce7c305aca9c99bfd448c978da48169109bf93

you can check it out using:

git checkout ddce7c305aca9c99bfd448c978da48169109bf93

**NOTE**: All commands below should be run from within an ssh session on the node1 created using
```bash
vagrant ssh node1
```

No need to precede each command by ```bash vagrant ssh node1 -c '<cmd>'``` as done below (only necessary in this notebook format)

#### Running the Guestbook example

The following is all taken from

https://github.com/kubernetes/kubernetes/tree/master/examples/guestbook

and this specific revision at:
https://github.com/kubernetes/kubernetes/tree/ddce7c305aca9c99bfd448c978da48169109bf93/examples/guestbook


In [10]:
vagrant ssh node1 -c "git clone https://github.com/GoogleCloudPlatform/kubernetes.git""

Cloning into 'kubernetes'...
remote: Counting objects: 367224, done.
remote: Compressing objects: 100% (5/5), done.
Receiving objects: 100% (367224/367224), 337.70 MiB | 4.78 MiB/s, done.
remote: Total 367224 (delta 0), reused 0 (delta 0), pack-reused 367219
Resolving deltas: 100% (241123/241123), done.



## Check the Kubernetes cluster status

By running the *kubectl cluster-info* command we should obtain output similar to below

In [12]:
vagrant ssh node1 -c "kubectl cluster-info"

Kubernetes master is running at http://localhost:8080



### Start the guestbook application - Quick Start

Below are the contents of the yaml file we will use to start the guestbook application under Kubernetes:

In [14]:
vagrant ssh node1 -c "cat kubernetes/examples/guestbook/all-in-one/guestbook-all-in-one.yaml"

apiVersion: v1
kind: Service
metadata:
  name: redis-master
  labels:
    app: redis
    tier: backend
    role: master
spec:
  ports:
    # the port that this service should serve on
  - port: 6379
    targetPort: 6379
  selector:
    app: redis
    tier: backend
    role: master
---
apiVersion: extensions/v1beta1
kind: Deployment
metadata:
  name: redis-master
  # these labels can be applied automatically
  # from the labels in the pod template if not set
  # labels:
  #   app: redis
  #   role: master
  #   tier: backend
spec:
  # this replicas value is default
  # modify it according to your case
  replicas: 1
  # selector can be applied automatically
  # from the labels in the pod template if not set
  # selector:
  #   matchLabels:
  #     app: guestbook
  #     role: master
  #     tier: backend
  template:
    metadata:
      labels:
        app: redis
        role: master
        tier: backend
    spec:
      containers:
      - name: master
        image: gcr.io/google_contai

In [13]:
vagrant ssh node1 -c "cd kubernetes; kubectl create -f examples/guestbook/all-in-one/guestbook-all-in-one.yaml"

service "redis-master" created
deployment "redis-master" created
service "redis-slave" created
deployment "redis-slave" created
service "frontend" created
deployment "frontend" created



vagrant ssh node1 -c "kubectl get services"

In [15]:
vagrant ssh node1 -c "kubectl get services"

NAME           CLUSTER-IP       EXTERNAL-IP   PORT(S)    AGE
frontend       10.254.225.118   <none>        80/TCP     8m
kubernetes     10.254.0.1       <none>        443/TCP    25m
redis-master   10.254.116.79    <none>        6379/TCP   8m
redis-slave    10.254.139.216   <none>        6379/TCP   8m



In [23]:
vagrant ssh node1 -c "curl http://10.254.225.118:80"

<html ng-app="redis">
  <head>
    <title>Guestbook</title>
    <link rel="stylesheet" href="//netdna.bootstrapcdn.com/bootstrap/3.1.1/css/bootstrap.min.css">
    <script src="https://ajax.googleapis.com/ajax/libs/angularjs/1.2.12/angular.min.js"></script>
    <script src="controllers.js"></script>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/angular-ui-bootstrap/0.13.0/ui-bootstrap-tpls.js"></script>
  </head>
  <body ng-controller="RedisCtrl">
    <div style="width: 50%; margin-left: 20px">
      <h2>Guestbook</h2>
    <form>
    <fieldset>
    <input ng-model="msg" placeholder="Messages" class="form-control" type="text" name="input"><br>
    <button type="button" class="btn btn-primary" ng-click="controller.onRedis()">Submit</button>
    </fieldset>
    </form>
    <div>
      <div ng-repeat="msg in messages track by $index">
        {{msg}}
      </div>
    </div>
    </div>
  </body>
</html>



In [25]:
vagrant ssh node1 -c "cd kubernetes; kubectl delete -f examples/guestbook/all-in-one/guestbook-all-in-one.yaml"

service "redis-master" deleted
deployment "redis-master" deleted
service "redis-slave" deleted
deployment "redis-slave" deleted
service "frontend" deleted
deployment "frontend" deleted



In [26]:
vagrant ssh node1 -c "kubectl get services"

NAME         CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
kubernetes   10.254.0.1   <none>        443/TCP   1h



#### Step1: Start the Redis Master

We will start the Redis Master using the deployment file listed below

In [33]:
vagrant ssh node1 -c "cat kubernetes/examples/guestbook/redis-master-service.yaml"

apiVersion: v1
kind: Service
metadata:
  name: redis-master
  labels:
    app: redis
    role: master
    tier: backend
spec:
  ports:
    # the port that this service should serve on
  - port: 6379
    targetPort: 6379
  selector:
    app: redis
    role: master
    tier: backend



In [35]:
vagrant ssh node1 -c "kubectl create -f kubernetes/examples/guestbook/redis-master-service.yaml"

service "redis-master" created



We can now see below that the *'redis-master'* service has been created:

In [36]:
vagrant ssh node1 -c "kubectl get services"

NAME           CLUSTER-IP       EXTERNAL-IP   PORT(S)    AGE
kubernetes     10.254.0.1       <none>        443/TCP    1h
redis-master   10.254.193.146   <none>        6379/TCP   6s



In [38]:
vagrant ssh node1 -c "kubectl create -f kubernetes/examples/guestbook/redis-master-deployment.yaml"

deployment "redis-master" created



In [39]:
vagrant ssh node1 -c "kubectl get deployments"

NAME           DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
redis-master   1         1         1            1           7s



In [40]:
vagrant ssh node1 -c "kubectl get pods"

NAME                            READY     STATUS    RESTARTS   AGE
redis-master-2353460263-bd47e   1/1       Running   0          29s



In [41]:
vagrant ssh node1 -c "kubectl get pods --all-namespaces"

NAMESPACE   NAME                            READY     STATUS    RESTARTS   AGE
default     redis-master-2353460263-bd47e   1/1       Running   0          1m



In [42]:
vagrant ssh node1 -c "kubectl describe pods redis-master-2353460263-bd47e"

Name:		redis-master-2353460263-bd47e
Namespace:	default
Node:		node1/10.0.2.15
Start Time:	Mon, 31 Oct 2016 12:54:49 +0000
Labels:		app=redis,pod-template-hash=2353460263,role=master,tier=backend
Status:		Running
IP:		172.17.96.2
Controllers:	ReplicaSet/redis-master-2353460263
Containers:
  master:
    Container ID:	docker://0d00c2bb0b2c572603ac25c3d3e94a746c5fbe8a81007d18f55b1d0af0277c56
    Image:		gcr.io/google_containers/redis:e2e
    Image ID:		docker://sha256:e5e67996c442f903cda78dd983ea6e94bb4e542950fd2eba666b44cbd303df42
    Port:		6379/TCP
    QoS Tier:
      cpu:	Burstable
      memory:	Burstable
    Requests:
      cpu:		100m
      memory:		100Mi
    State:		Running
      Started:		Mon, 31 Oct 2016 12:54:50 +0000
    Ready:		True
    Restart Count:	0
    Environment Variables:
Conditions:
  Type		Status
  Ready 	True 
No volumes.
Events:
  FirstSeen	LastSeen	Count	From			SubobjectPath		Type		Reason			Message
  ---------	--------	-----	----			-------------		--------	------			

In [43]:
vagrant ssh node1 -c "kubectl logs redis-master-2353460263-bd47e"

                _._                                                  
           _.-``__ ''-._                                             
      _.-``    `.  `_.  ''-._           Redis 2.8.19 (00000000/0) 64 bit
  .-`` .-```.  ```\/    _.,_ ''-._                                   
 (    '      ,       .-`  | `,    )     Running in stand alone mode
 |`-._`-...-` __...-.``-._|'` _.-'|     Port: 6379
 |    `-._   `._    /     _.-'    |     PID: 1
  `-._    `-._  `-./  _.-'    _.-'                                   
 |`-._`-._    `-.__.-'    _.-'_.-'|                                  
 |    `-._`-._        _.-'_.-'    |           http://redis.io        
  `-._    `-._`-.__.-'_.-'    _.-'                                   
 |`-._`-._    `-.__.-'    _.-'_.-'|                                  
 |    `-._`-._        _.-'_.-'    |                                  
  `-._    `-._`-.__.-'_.-'    _.-'                                   
      `-._    `-.__.-'    _.-'                               

#### Step2: Start the Redis Slave

We will start the Redis Slave using the deployment file listed below

In [44]:
vagrant ssh node1 -c "cat kubernetes/examples/guestbook/redis-slave-service.yaml"

apiVersion: v1
kind: Service
metadata:
  name: redis-slave
  labels:
    app: redis
    role: slave
    tier: backend
spec:
  ports:
    # the port that this service should serve on
  - port: 6379
  selector:
    app: redis
    role: slave
    tier: backend



In [45]:
vagrant ssh node1 -c "kubectl create -f kubernetes/examples/guestbook/redis-slave-service.yaml"

service "redis-slave" created



In [46]:
vagrant ssh node1 -c "kubectl get services"

NAME           CLUSTER-IP       EXTERNAL-IP   PORT(S)    AGE
kubernetes     10.254.0.1       <none>        443/TCP    2h
redis-master   10.254.193.146   <none>        6379/TCP   17m
redis-slave    10.254.110.110   <none>        6379/TCP   50s



In [48]:
vagrant ssh node1 -c "kubectl create -f kubernetes/examples/guestbook/redis-slave-deployment.yaml"

deployment "redis-slave" created



In [49]:
vagrant ssh node1 -c "kubectl get deployments"

NAME           DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
redis-master   1         1         1            1           10m
redis-slave    2         2         2            2           4s



In [50]:
vagrant ssh node1 -c "kubectl get services"

NAME           CLUSTER-IP       EXTERNAL-IP   PORT(S)    AGE
kubernetes     10.254.0.1       <none>        443/TCP    2h
redis-master   10.254.193.146   <none>        6379/TCP   18m
redis-slave    10.254.110.110   <none>        6379/TCP   2m



In [51]:
vagrant ssh node1 -c "kubectl get pods"

NAME                            READY     STATUS    RESTARTS   AGE
redis-master-2353460263-bd47e   1/1       Running   0          10m
redis-slave-1691881626-1okos    1/1       Running   0          28s
redis-slave-1691881626-1u22e    1/1       Running   0          28s



#### Start the Guestbook frontend:

In [52]:
vagrant ssh node1 -c "cat kubernetes/examples/guestbook/all-in-one/frontend.yaml"

apiVersion: v1
kind: Service
metadata:
  name: frontend
  labels:
    app: guestbook
    tier: frontend
spec:
  # if your cluster supports it, uncomment the following to automatically create
  # an external load-balanced IP for the frontend service.
  # type: LoadBalancer
  ports:
    # the port that this service should serve on
  - port: 80
  selector:
    app: guestbook
    tier: frontend
---
apiVersion: extensions/v1beta1
kind: Deployment
metadata:
  name: frontend
  # these labels can be applied automatically
  # from the labels in the pod template if not set
  # labels:
  #   app: guestbook
  #   tier: frontend
spec:
  # this replicas value is default
  # modify it according to your case
  replicas: 3
  # selector can be applied automatically
  # from the labels in the pod template if not set
  # selector:
  #   matchLabels:
  #     app: guestbook
  #     tier: frontend
  template:
    metadata:
      labels:
        app: guestbook
        tier: frontend
    spec:
      containers

In [53]:
vagrant ssh node1 -c "kubectl create -f kubernetes/examples/guestbook/all-in-one/frontend.yaml"

service "frontend" created
deployment "frontend" created



In [54]:
vagrant ssh node1 -c "kubectl get services"

NAME           CLUSTER-IP       EXTERNAL-IP   PORT(S)    AGE
frontend       10.254.220.26    <none>        80/TCP     2m
kubernetes     10.254.0.1       <none>        443/TCP    2h
redis-master   10.254.193.146   <none>        6379/TCP   23m
redis-slave    10.254.110.110   <none>        6379/TCP   7m



In [55]:
vagrant ssh node1 -c "kubectl get deployments"

NAME           DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
frontend       3         3         3            1           3m
redis-master   1         1         1            1           15m
redis-slave    2         2         2            2           5m



In [56]:
vagrant ssh node1 -c "kubectl get pods"

NAME                            READY     STATUS    RESTARTS   AGE
frontend-440143558-3xe8u        1/1       Running   0          3m
frontend-440143558-5e211        0/1       Pending   0          3m
frontend-440143558-qngwf        0/1       Pending   0          3m
redis-master-2353460263-bd47e   1/1       Running   0          16m
redis-slave-1691881626-1okos    1/1       Running   0          5m
redis-slave-1691881626-1u22e    1/1       Running   0          5m



In [57]:
vagrant ssh node1 -c "kubectl get pods -L tier"

NAME                            READY     STATUS    RESTARTS   AGE       TIER
frontend-440143558-3xe8u        1/1       Running   0          3m        frontend
frontend-440143558-5e211        0/1       Pending   0          3m        frontend
frontend-440143558-qngwf        0/1       Pending   0          3m        frontend
redis-master-2353460263-bd47e   1/1       Running   0          16m       backend
redis-slave-1691881626-1okos    1/1       Running   0          6m        backend
redis-slave-1691881626-1u22e    1/1       Running   0          6m        backend



#### Cleanup

In [60]:
vagrant ssh node1 -c "kubectl delete deployments,services -l 'app in (redis, guestbook)'"

deployment "frontend" deleted
deployment "redis-master" deleted
deployment "redis-slave" deleted
service "frontend" deleted
service "redis-master" deleted
service "redis-slave" deleted



In [61]:
vagrant ssh node1 -c "kubectl get services"

NAME         CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
kubernetes   10.254.0.1   <none>        443/TCP   2h

